# What is nnU-Net?

**nnU-Net** (no-new-U-Net) is a **fully automated deep-learning framework for biomedical image segmentation**.
It is not just a model.
It is a **system** that:

* analyzes your dataset,
* configures the entire pipeline automatically,
* trains several models,
* ensembles them,
* and gives you a strong baseline without manual tuning.

Its key idea:
**Automate everything that experts usually design manually** (preprocessing, architecture, patch size, data augmentation, postprocessing, etc.)

---

# Core Idea

Given a new medical dataset (CT, MRI, multi-modal, anisotropic, 2D, 3D…), nnU-Net:

1. **Reads dataset properties** (voxel spacing, anisotropy, image size, number of channels)
2. **Chooses the architecture variant** (2D U-Net, 3D U-Net, 3D U-Net cascade)
3. **Chooses patch size, batch size, number of filters**
4. **Applies optimal preprocessing**
5. **Applies powerful data augmentation**
6. **Trains using standardized training rules**
7. **Performs automatic postprocessing**
8. **Ensembles all models**

And it works extremely well across many medical challenges (Decathlon, KiTS, BraTS, PROMISE12, etc.).

---

# Pipeline Overview

## 1. Dataset Analysis (Automated)

nnU-Net reads the dataset metadata:

* image spacings
* voxel anisotropy
* image shapes
* modalities (CT/MRI channels)
* number of segmentation classes

Based on this, it decides whether the dataset is:

* best handled with **2D U-Net**
* **3D full-resolution U-Net**
* **3D low-resolution + cascade model**

Example:
If voxel spacing has large anisotropy (slice thickness ≫ in-plane resolution), nnU-Net favors **2D** or **3D large-spacing** models.

---

## 2. Automated Architecture Construction

It builds a U-Net variant with:

* number of stages chosen based on memory limits
* convolution kernel sizes chosen based on anisotropy
* dynamic number of feature maps
* skip connections
* instance normalization
* leaky ReLU or GELU depending on version

Key:
Architecture is **not manually designed** – it adapts to dataset geometry and GPU memory.

---

## 3. Automated Preprocessing

### a. Intensity Normalization

* **Z-score** for MRI
* **ct-clipping + normalization** for CT (based on HU)

### b. Resampling

All images are resampled to a **target spacing** chosen automatically.
Preserves anatomical correctness.

### c. Cropping / foreground detection

Reduces empty space for faster training.

---

## 4. Automated Training Configuration

nnU-Net chooses:

* patch size
* batch size
* sampling strategy (foreground oversampling)
* optimizer: **SGD with Nesterov**
* learning rate schedule: **poly LR**

$$ lr = lr_0 \left(1 - \frac{iter}{iter_{max}} \right)^{0.9} $$

---

## 5. Automated Data Augmentation

A strong augmentation pipeline including:

* elastic deformations
* gamma transformations
* gaussian noise
* rotations & flipping
* mirroring
* scale & intensity shift

Everything is selected depending on dimensionality (2D/3D).

---

## 6. Model Types nnU-Net Trains

nnU-Net trains **three models**:

1. **2D U-Net**
2. **3D full-resolution U-Net**
3. **3D U-Net cascade** (coarse → fine)

Not all datasets use all three.
It chooses automatically.

---

## 7. Ensembling

At inference time, predictions from the different models are **ensembled** to boost performance.

---

## 8. Automated Postprocessing

nnU-Net looks at validation errors and decides:

* removing small connected components,
* class-wise postprocessing,
* refining multi-organ predictions.

---

# Why nnU-Net Works So Well

It removes manual guesswork and standardizes all components.

Key aspects:

1. **Universality** – works out of the box for almost any 3D/2D medical segmentation task.
2. **Robust preprocessing** – automatic spacing/patch decisions.
3. **Heavy augmentation** – prevents overfitting on small medical datasets.
4. **Strong ensembling** – boosts performance.
5. **No architecture-hacking needed** – prevents overengineering.

---

# Architecture Overview (U-Net Style)

Prototype architecture:

* Encoder:
  Conv → Norm → Nonlinearity → Downsample
* Decoder:
  Upsample → Concatenate skip → Conv blocks
* Output:
  1×1 convolution to class logits

But nnU-Net adjusts:

* depth
* number of filters
* kernel sizes
* spacing handling

All from dataset analysis.

---

# When Should You Use nnU-Net?

nnU-Net is ideal when:

* You have limited data (typical medical imaging)
* You want a **baseline** that is very strong
* You want high performance without hyperparameter search
* You work with CT, MRI, microscopy, 3D volumes
* You want to avoid huge architecture engineering time

You should NOT use it when:

* You need non-U-Net architectures (transformers, SAM-style)
* You need extremely low-latency inference
* You must control every part of the pipeline manually

---

# Summary

**nnU-Net = automatic, adaptive, standardized biomedical segmentation system.**

It configures:

* preprocessing
* architecture
* training
* augmentation
* postprocessing
* ensembling

and provides **state-of-the-art** results on new datasets without manual tuning.
